# Introduction 

The Chesapeake Bay Program [DataHub](https://datahub.chesapeakebay.net/Home) contains many datasets for the Chesapeake Bay. 

The Water Quality Data is still updating and measures many field and lab parameters including: phosphorus, nitrogen, carbon, various other lab parameters (suspended solids, disolved solids, chlorophyll-a, alkalinkity, etc), dissolved oxygen, pH, salinity, turbitity, water temperature, and climate condition. See [Guide to Using Chesapeake Bay Program Water Quality Monitoring Data](https://d18lev1ok5leia.cloudfront.net/chesapeakebay/documents/wq_data_userguide_10feb12_mod.pdf) for more information.

There is also a [DataHub API](https://datahub.chesapeakebay.net/API) which we will used to access the data from July 29, 2004 through July 29, 2024. It is a lot of data. 

In [11]:
import pandas as pd
import numpy as np
import json

In [24]:
water = pd.read_csv('../../data/plank_ChesapeakeWaterQuality.csv')

/var/folders/rl/kqt6tbv90l9_pwc4927vdb340000gn/T/ipykernel_83404/3917417675.py:1: DtypeWarning: Columns (25,30,34) have mixed types. Specify dtype option on import or set low_memory=False.
  water = pd.read_csv('../../data/plank_ChesapeakeWaterQuality.csv')


I forgot to set the index to false, and this is already a big file. Let's just drop that column. Also, `BiasPC` only has a value in one row, so lets drop that. `FieldActivityEventType` is only in 6 rows.

In [25]:
water = water.drop(columns=[water.columns[0],'BiasPC','FieldActivityEventType'])

# Analyzing columns and creating dictionaries

First, let's see what columns are we have and some stats.

In [26]:
water.columns

Index(['CBSeg2003', 'EventId', 'Cruise', 'Program', 'Project', 'Agency',
       'Source', 'Station', 'SampleDate', 'SampleTime', 'TotalDepth',
       'UpperPycnocline', 'LowerPycnocline', 'Depth', 'Layer', 'SampleType',
       'SampleReplicateType', 'Parameter', 'Qualifier', 'MeasureValue', 'Unit',
       'Method', 'Lab', 'Problem', 'Details', 'Latitude', 'Longitude',
       'TierLevel', 'AirTemp', 'WindSpeed', 'WindDirection', 'PrecipType',
       'TideStage', 'WaveHeight', 'CloudCover'],
      dtype='object')

In [27]:
water.shape

(2173439, 35)

We will create a dictionary between the parameters and their units. We will manually create the initial dictionary between the parameter and its meaning, since they are in a PDF.

In [28]:
water_parameters = water['Parameter'].unique()

print(water_parameters)

['CHLA' 'DIN' 'DO' 'DON' 'DOP' 'KD' 'NH4F' 'NO23F' 'NO2F' 'NO3F' 'PC' 'PH'
 'PHEO' 'PN' 'PO4F' 'PP' 'SALINITY' 'SECCHI' 'SIF' 'SIGMA_T' 'SPCOND'
 'TDN' 'TDP' 'TN' 'TON' 'TP' 'TSS' 'WTEMP' 'VSS' nan 'DOC' 'PIP' 'FSS'
 'TURB_NTU' 'DO_SAT_P']


In [21]:
# Create the initial dictionary with parameter and its meaning
param_to_meaning = {'CHLA': 'Chlorophyll-a', 
                    'DIN': 'Dissolved inorganic nitrogen', 
                    'DO': 'Dissolved oxygen', 
                    'DON': 'Dissolved organic nitrogen', 
                    'DOP': 'Dissolved organic phosphorus',
                    'KD': 'Light attenuation',
                    'NH4F': 'Ammonium (filtered)',
                    'NO23F': 'Nitrite + nitrate (filtered)',
                    'NO2F': 'Nitrite ( filtered)', 'NO3F': 'Nitrite ( filtered)',
                    'PC': 'Particulate organic carbon',
                    'PH' : 'pH','PHEO': 'pheophytin',
                    'PN': 'Particulate Organic Nitrogen and Particulate Nitrogen', 
                    'PO4F': 'Orthophosphorus (filtered)', 'PP': 'Particulate phosphorus','SALINITY': 'Salinity', 
                    'SECCHI': 'Secchi disk depth',
                    'SIF': 'Silica (filtered)', 
                    'SIGMA_T': 'Specific gravity',
                    'SPCOND': 'Specific conductivity',
                    'TDN': 'Total dissolved nitrogen',
                    'TDP': 'Total dissolved phosphorus', 
                    'TN': 'Total nitrogen',
                    'TON': 'Total organic nitrogen',
                    'TP': 'Total phosphorus',
                    'TSS': 'Total suspended solids', 
                    'WTEMP': 'Water temperature',
                    'VSS': '?',
                    'DOC': 'Dissolved organic carbon',
                    'PIP': 'Particulate phosphorus',
                    'FSS': 'Fixed suspended solids','TURB_NTU': 'Turbidity: nephelometric method',
                    'DO_SAT_P' : 'Dissolved oxygen relative To theoretical value At saturation'}

Create dictionary for meaning and units

In [30]:
water_param_dict = {param: {'Units': unit, 'Type': ""} for param, unit in zip(water['Parameter'], water['Unit'])}

# Update the dictionary with the meanings
for param in water_param_dict:
    if param in param_to_meaning:
        water_param_dict[param]['Type'] = param_to_meaning[param]

In [19]:
water = water.drop(columns='Unit')

And add to the JSON dictionary.

In [31]:
# Import from JSON file
with open('../../data/plank_CBPparam_dict.json', 'r') as f:
    benthic_param_dict = json.load(f)

new_param_dict = {**benthic_param_dict, **water_param_dict}

with open('../../data/plank_CBPparam_dict.json', 'w') as f:
    json.dump(new_param_dict, f)


# Turn parameters into columns

In [112]:
def parameter_to_columns(dataframe,columns_to_group):
    # Reset index to use row numbers as the index
    df_reset = dataframe.reset_index(drop=True)

    # Pivot the DataFrame while preserving non-pivoted columns
    df_pivoted = df_reset.pivot_table(index=df_reset.index, columns='Parameter', values='MeasureValue', aggfunc='first')

    # Combine pivoted result with the original DataFrame columns not involved in the pivot
    df_pivoted = df_reset.drop(columns=['Parameter','MeasureValue']).join(df_pivoted)
    
    # Create a copy of the DataFrame for processing
    df_processed = df_pivoted.copy()

    # Create a unique identifier for each group based on the columns to match
    df_processed['UniqueID'] = df_processed[columns_to_group].astype(str).agg('-'.join, axis=1)

    # Group by the unique identifier
    df_combined = df_processed.groupby('UniqueID', as_index=False).first()

    # Drop the UniqueID column and remove duplicates
    df_clean = df_combined.drop(columns='UniqueID').drop_duplicates()

    
    return df_clean

Let's get the columns to group by calling the APi for one year of monitoring data. Some of the columns were empty, so we will intersect the monitor data columns with our water data columns.

In [113]:
monitor_data = pd.read_csv('https://datahub.chesapeakebay.net/api.CSV/WaterQuality/MonitorEvent/8-5-2023/8-6-2024/2,4,6/12,13,15,35,36,2,3,7,33,34,23,24/CBSeg2003/10,11,12,13,14,15,16,17,28,49,84')

In [114]:
columns_to_group = monitor_data.columns.intersection(water.columns)

Let's apply the function!

In [115]:
water_clean = parameter_to_columns(water,columns_to_group)

Let's see how big the dataframe is now!!

In [116]:
water_clean.shape

(39490, 66)

In [117]:
water_clean['EventId'].nunique()

20140

In [118]:
water_clean.to_csv('../../data/plank_ChesapeakeWaterQuality_pivoted.csv',index=False)